## PDF Query Using Langchain

In [ ]:
!pip install langchain
!pip install openai
!pip install PyPDF2
!pip install faiss-cpu
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.6/983.6 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.3/366.3 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.7 MB/s eta 0:00:00


In [ ]:
!pip install langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.4 MB/s eta 0:00:00


In [ ]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

In [ ]:
# Add your own openai api
import os
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
# provide the path of  pdf file/files.
pdfreader = PdfReader('/content/pdf/Budget.pdf')

In [ ]:
from typing_extensions import Concatenate
# read text from pdf
raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

In [ ]:
raw_text

' \n \nSEMESTER -3  \n \n \n \n \n \n \n \n1 \n CODE  COURSE NAME  CATEGORY  L T P CREDIT  \n20MCA201  DATA SCIENCE & \nMACHINE LEARNING  CORE  3 1 0 4 \nPreamble: This is an introductory course on data science and basic concepts behind various    \nmachine learning techniques. Machine learning is the study of adaptive computational systems \nthat improve their performance with experience. At the end of the course the students should \nbe able to design and implement machine learning solutions to classification, regression, and \nclustering problems and to evaluate an d interpret the results of the algorithms.  \nPrerequisite:  Probability and Statistics, Linear Algebra, Programming in Python/R.  \nCourse Outcomes: After the completion of the course the student will be able to:  \nCO No.  Course Outcome (CO)  Bloom\'s Category \nLevel  \nCO 1  Discuss  the fundamental concepts of data science and data \nvisualization techniques . Level 2:  \nUnderstand  \nCO 2  Explain  the basics of m

In [ ]:
# We need to split the text using Character Text Split such that it sshould not increse token size
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [ ]:
len(texts)

303

In [ ]:
!pip install -U langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 2.3 MB/s eta 0:00:00


In [ ]:
# Download embeddings from OpenAI
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [ ]:
document_search = FAISS.from_texts(texts, embeddings)

In [ ]:
document_search


In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [ ]:
chain = load_qa_chain(OpenAI(), chain_type="stuff")

In [ ]:
query = "Vision for Amrit Kaal"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

' Our vision for the Amrit Kaal includes technology-driven and knowledge-based economy with strong public finances, and a robust financial sector. To achieve this, Jan Bhagidari through Sabka Saath Sabka Prayas is essential. The economic agenda for achieving this vision focuses on three things: first, facilitating ample opportunities for citizens, especially the youth, to fulfil their aspirations; second, providing strong impetus to growth and job creation; and third, strengthening macro-economic stability.'

In [ ]:
query = "How much the agriculture target will be increased to and what the focus will be"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

' The agriculture credit target will be increased to ` 20 lakh crore with focus on animal husbandry, dairy and fisheries.'